In [10]:
import functools

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib as plt

import optuna

from sklearn import datasets
import sklearn.datasets
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import functools
import xgboost as xgb
import psutil
import time

In [11]:
df = pd.read_csv('../data/dataset/DTX_data_002.csv')
df

,금연성공여부,연령,흡연시작연령,하루흡연량,담배종류_1,담배종류_2,담배종류_4,담배종류_5,담배종류_6,담배종류_7,...,FTND_1,FTND_2,FTND_3,FTND_4,FTND_5,FTND_6,FTND_7,FTND_8,FTND_9,FTND_10
0,0,52,25,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,32,17,4,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1,41,21,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,60,18,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,28,19,2,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0,33,17,2,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1051,0,24,20,2,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1052,0,49,30,2,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1053,1,55,30,3,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
X = df.drop(['금연성공여부'],axis=1)
y = df['금연성공여부']

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.1, shuffle=True,  random_state=34)

In [13]:
def opt(X_train, y_train, X_test, y_test, trial):
    #param_list
    n_estimators = trial.suggest_int('n_estimators', 0, 100)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 20)
    learning_rate = trial.suggest_discrete_uniform('learning_rate', 0.01, 0.1, 0.01)
    scale_pos_weight = trial.suggest_int('scale_pos_weight', 1, 100)
    subsample = trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1)
    colsample_bytree = trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1)

    xgboost_tuna = xgb.XGBClassifier(
        random_state=42,
        n_estimators = n_estimators,
        #max_depth = max_depth,
        # min_child_weight = min_child_weight,
        # learning_rate = learning_rate,
        #scale_pos_weight = scale_pos_weight,
        # subsample = subsample,
        # colsample_bytree = colsample_bytree,
        use_label_encoder = False
    )
    xgboost_tuna.fit(X_train, y_train)
    tuna_pred_test = xgboost_tuna.predict(X_test)

    return (1.0 - (accuracy_score(y_test, tuna_pred_test)))

In [14]:
study = optuna.create_study()
study.optimize(functools.partial(opt, X_train, y_train, X_eval, y_eval), n_trials=100)

[I 2022-05-19 13:28:07,326] A new study created in memory with name: no-name-0a8a2800-8bd2-492d-915a-c1eb09798252
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[13:28:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-05-19 13:28:07,571] Trial 0 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 86, 'max_depth': 15, 'min_child_weight': 11, 'learning_rate': 0.02, 'scale_pos_weight': 2, 'subsample': 0.7, 'colsample_bytree': 0.7}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
[I 2022-05-19 13:28:07,717] Trial 1 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 23, 'max_depth': 8, 'min_child_weight': 9, 'learning_rate': 0.08, 'scale_pos_weight': 24, 'subsample': 0.8, 'colsample_bytree': 0.8}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning

[13:28:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:28:07] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-05-19 13:28:07,888] Trial 2 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 33, 'max_depth': 4, 'min_child_weight': 4, 'learning_rate': 0.09, 'scale_pos_weight': 79, 'subsample': 0.8, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
[I 2022-05-19 13:28:08,032] Trial 3 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 10, 'max_depth': 17, 'min_child_weight': 13, 'learning_rate': 0.08, 'scale_pos_weight': 3, 'subsample': 0.9, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning

...

[I 2022-05-19 13:28:23,308] Trial 95 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 15, 'max_depth': 19, 'min_child_weight': 13, 'learning_rate': 0.09, 'scale_pos_weight': 53, 'subsample': 0.9, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
[I 2022-05-19 13:28:23,482] Trial 96 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 26, 'max_depth': 18, 'min_child_weight': 13, 'learning_rate': 0.09, 'scale_pos_weight': 50, 'subsample': 0.8, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWa

[13:28:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:28:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-05-19 13:28:23,675] Trial 97 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 18, 'max_depth': 19, 'min_child_weight': 14, 'learning_rate': 0.08, 'scale_pos_weight': 59, 'subsample': 0.9, 'colsample_bytree': 0.9}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
[I 2022-05-19 13:28:23,832] Trial 98 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 54, 'max_depth': 18, 'min_child_weight': 11, 'learning_rate': 0.09999999999999999, 'scale_pos_weight': 53, 'subsample': 0.9, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.4528301886792453.
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.p

[13:28:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:28:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[I 2022-05-19 13:28:24,018] Trial 99 finished with value: 0.4528301886792453 and parameters: {'n_estimators': 28, 'max_depth': 12, 'min_child_weight': 16, 'learning_rate': 0.08, 'scale_pos_weight': 61, 'subsample': 0.8, 'colsample_bytree': 0.5}. Best is trial 0 with value: 0.4528301886792453.


In [15]:
study.best_params

{'n_estimators': 86,
 'max_depth': 15,
 'min_child_weight': 11,
 'learning_rate': 0.02,
 'scale_pos_weight': 2,
 'subsample': 0.7,
 'colsample_bytree': 0.7}

In [16]:
xgb_model = xgb.XGBClassifier(**study.best_params)
xgb_model.fit(X_train, y_train)

/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/aidev_yyc_mac/opt/anaconda3/envs/yyc/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[13:28:24] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7,
              enable_categorical=False, gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=15,
              min_child_weight=11, missing=nan, monotone_constraints='()',
              n_estimators=86, n_jobs=8, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=2,
              subsample=0.7, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [17]:
y_pre = xgb_model.predict(X_eval)
print(y_pre)
y_pred_probs = xgb_model.predict_proba(X_eval)[:,1]

[0 1 0 1 1 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 1 1 0 1 1
 1 1 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1
 1 0 0 0 0 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 0 0 1 0 0 1 1 1 0 0 1 1]


In [18]:

p = precision_score(y_eval, y_pre)
print('Precission :' , p)
r = recall_score(y_eval, y_pre)
print('recall :',r)
f1 = f1_score(y_eval, y_pre)
print('f1_score :',f1)
accuracy = accuracy_score( y_eval, y_pre)
print('accuracy :' ,accuracy)

Precission : 0.3770491803278688
recall : 0.6388888888888888
f1_score : 0.4742268041237113
accuracy : 0.5188679245283019
